In [1]:
import tensorflow as tf
import numpy as np
import os, pdb
import cv2
import numpy as np
import random as rn
import threading
import time
from sklearn import metrics
import utils
global n_classes
import triplet_loss as tri
import os.path

n_classes = 2


def activation(x,name="activation"):
    return tf.nn.swish(x, name=name)
    
def conv2d(name, l_input, w, b, s, p):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b

    return l_input

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, training=isTraining, name="bn"+name)

def initializer(in_filters, out_filters, name, k_size=3):
    w1 = tf.get_variable(name+"W", [k_size, k_size, in_filters, out_filters], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable(name+"B", [out_filters], initializer=tf.truncated_normal_initializer())
    return w1, b1
  
def residual_block(in_x, in_filters, out_filters, stride, isDownSampled, name, isTraining, k_size=3):
    global ema_gp
    # first convolution layer
    if isDownSampled:
      in_x = tf.nn.avg_pool(in_x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
      
    x = batchnorm(in_x, isTraining, name=name+'FirstBn')
    x = activation(x)
    w1, b1 = initializer(in_filters, in_filters, name+"first_res", k_size=k_size)
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")

    # second convolution layer
    x = batchnorm(x, isTraining, name=name+'SecondBn')
    x = activation(x)
    w2, b2 = initializer(in_filters, out_filters, name+"Second_res",k_size=k_size)
    x = conv2d(name+'r2', x, w2, b2, 1, "SAME")
    
    if in_filters != out_filters:
        difference = out_filters - in_filters
        left_pad = difference // 2
        right_pad = difference - left_pad
        identity = tf.pad(in_x, [[0, 0], [0, 0], [0, 0], [left_pad, right_pad]])
        return x + identity
    else:
        return in_x + x

      
def ResNet(_X, isTraining):
    global n_classes
    w1 = tf.get_variable("initWeight", [7, 7, 3, 64], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable("initBias", [64], initializer=tf.truncated_normal_initializer())
    initx = conv2d('conv1', _X, w1, b1, 4, "VALID")
    
    filters_num = [64,96,128]
    block_num = [2,4,3]
    l_cnt = 1
    x = initx
    for i in range(len(filters_num)):
      for j in range(block_num[i]):
          
          if ((j==block_num[i]-1) & (i<len(filters_num)-1)):
            x = residual_block(x, filters_num[i], filters_num[i+1], 2, True, 'ResidualBlock%d'%(l_cnt), isTraining)
            print('[L-%d] Build %dth connection layer %d from %d to %d channels' % (l_cnt, i, j, filters_num[i], filters_num[i+1]))
          else:
            x = residual_block(x, filters_num[i], filters_num[i], 1, False, 'ResidualBlock%d'%(l_cnt), isTraining)
            print('[L-%d] Build %dth residual block %d with %d channels' % (l_cnt,i, j, filters_num[i]))
          l_cnt +=1
    
    layer_33 = x
    x = initx
    
    for i in range(len(filters_num)):
      for j in range(block_num[i]):
          
          if ((j==block_num[i]-1) & (i<len(filters_num)-1)):
            x = residual_block(x, filters_num[i], filters_num[i+1], 2, True, 'Residual5Block%d'%(l_cnt), isTraining, k_size=5)
            print('[L-%d] Build %dth connection layer %d from %d to %d channels' % (l_cnt, i, j, filters_num[i], filters_num[i+1]))
          else:
            x = residual_block(x, filters_num[i], filters_num[i], 1, False, 'Residual5Block%d'%(l_cnt), isTraining, k_size=5)
            print('[L-%d] Build %dth residual block %d with %d channels' % (l_cnt,i, j, filters_num[i]))
          l_cnt +=1
    layer_55 = x
    print("Layer33's shape", layer_33.get_shape().as_list())
    print("Layer55's shape", layer_55.get_shape().as_list())

    x = tf.concat([layer_33, layer_55], 3)
    
    saliency = x
    x_shape = x.get_shape().as_list()
    dense1 = x_shape[1]*x_shape[2]*x_shape[3]
    W = tf.get_variable("featW", [dense1, 128], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("featB", [128], initializer=tf.truncated_normal_initializer())
    dense1 = tf.reshape(x, [-1, dense1])
    feat = tf.nn.softmax(tf.matmul(dense1, W) + b)
    
    with tf.variable_scope('Final'):
        x = batchnorm(x, isTraining, name='FinalBn')
        x = activation(x)
        wo, bo=initializer(filters_num[-1]*2, n_classes, "FinalOutput")
        x = conv2d('final', x, wo, bo, 1, "SAME")
        x=tf.reduce_mean(x, [1, 2])

        W = tf.get_variable("FinalW", [n_classes, n_classes], initializer=tf.truncated_normal_initializer())
        b = tf.get_variable("FinalB", [n_classes], initializer=tf.truncated_normal_initializer())

        out = tf.matmul(x, W) + b
                            

    return out, feat, saliency


#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=64):
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn
    
def setup_inputs(sess, filenames, training_img_dir, image_size=64, crop_size=64, isTest=False, batch_size=128):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
        
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)

#====================================Main function=================================
# os.mkdir('logs/pair')
TRAIN_WO_SPEC_GAN = 'progressGAN'
if not os.path.isdir('logs/pair/%s/'%(TRAIN_WO_SPEC_GAN)):
    os.mkdir('logs/pair/%s/'%(TRAIN_WO_SPEC_GAN))

batch_size = 64
display_step = 80
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-3                        # Learning rate start
tst = tf.placeholder(tf.bool)
iter = tf.placeholder(tf.int32)
print('GO!!')


# In[ ]:


# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "data/train_wo_%s.txt"%(TRAIN_WO_SPEC_GAN), "./", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "data/val_wo_%s.txt"%(TRAIN_WO_SPEC_GAN), "./", batch_size=1000,isTest=True)
print("Found %d training images, and %d validation images..." % (glen1, tlen1))

max_iter = glen1*15
print("Preparing the training model with learning rate = %.5f..." % (lr))


with tf.variable_scope("ResNet") as scope:
    pred, feat,_ = ResNet(train_data, True)
    scope.reuse_variables()
    valpred, _, saliency = ResNet(val_data, False)


  
with tf.name_scope('Triplet_loss'):
  margin = 0.4
  sialoss = tri.batch_hard_triplet_loss(train_labels, feat, margin, squared=False)
  
  
with tf.name_scope('Loss_and_Accuracy'):
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  with tf.control_dependencies(update_ops):
    t_vars=tf.trainable_variables() 
    #t_vars=[var for var in t_vars if 'Final']
    cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, var_list=t_vars)
    sia_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(sialoss)
  
  correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  correct_prediction2 = tf.equal(tf.argmax(valpred, 1), val_labels)
  accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
  
  
tf.summary.scalar("Triplet_loss", sialoss)
tf.summary.scalar('Loss', cost)
tf.summary.scalar('Training_Accuracy', accuracy)
tf.summary.scalar('Validation_Accuracy', accuracy2)
# In[ ]:


saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
step = 0

writer = tf.summary.FileWriter("logs/pair/%s/"%(TRAIN_WO_SPEC_GAN), sess.graph)
summaries = tf.summary.merge_all()

print("We are going to pretrain model susing ResNet based on triplet loss!!!")
start_lr = lr
while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-4) & (epoch1 >=3)):
        lr /= 10
    
    if epoch1 <=3:
        sess.run([sia_optimizer],  feed_dict={learning_rate: lr})
    else:
#         if start_lr == lr:
#             lr = lr /10
        sess.run([optimizer],  feed_dict={learning_rate: lr})
        
    if (step % 15000==1) & (step>15000):
        save_path = saver.save(sess, "checkpoints/tf_deepUD_tri_model_iter_%d_for_%s.ckpt" % (step,TRAIN_WO_SPEC_GAN))
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step>0 and step % display_step == 0:
        # calculate the loss
        loss, acc, summaries_string, sia_val = sess.run([cost, accuracy, summaries, sialoss])
        print("Iter=%d/epoch=%d, Loss=%.6f, Triplet loss=%.6f, Training Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, sia_val, acc, lr))
        writer.add_summary(summaries_string, step)
    
    if step>0 and (step % (display_step*20) == 0):
        rounds = tlen1 // 1000
        #pdb.set_trace()
        valacc=[]
        vis=[]
        tis=[]
        for k in range(rounds):
          a2, vi, ti = sess.run([accuracy2, tf.argmax(valpred, 1), val_labels])
          valacc.append(a2)
          vis.append(vi)
          tis.append(ti)
        tis = np.reshape(np.asarray(tis), [-1])
        vis = np.reshape(np.asarray(vis), [-1])
        precision=metrics.precision_score(tis, vis) 
        recall=metrics.recall_score(tis, vis)
        
        sal, valimg = sess.run([saliency, val_data])
        utils.batchsalwrite(valimg, sal, tis, vis, 'saliency_img/%s_Detected_'%(TRAIN_WO_SPEC_GAN))
        

        print("Iter=%d/epoch=%d, Validation Accuracy=%.6f, Precision=%.6f, Recall=%.6f" % (step*batch_size, epoch1 , np.mean(valacc), precision, recall))

  
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "checkpoints/tf_deepUD_tri_model_%s.ckpt" % (TRAIN_WO_SPEC_GAN))
print("Model saved in file: %s" % save_path)






GO!!
Preparing the training & validation data...
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Found 385198 training images, and 10000 validation images...
Preparing the training model with learning rate = 0.00100...
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
[L-1] Build 0th residual block 0 with 64 channels
[L-2] Build 0th connection layer 1 from 64 to 96 channels
[L-3] Build 1th residual block 0 with 96 channels
[L-4] Build 1th residual block 1 with 96 channels
[L-5] Build 1th residual block 2 with 96 channels
[L-6] Build 1th connection layer 3 from 96 to 128 channels
[L-7] Build 2th residual block 0 with 128 channels
[L-8] Build 2th residual block 1 with 128 channels
[L-9] Build 2th residual block 2 with 128 channels
[L-10

Iter=235520/epoch=0, Loss=14.223509, Triplet loss=1.814214, Training Accuracy=0.562500, lr=0.001000
Iter=240640/epoch=0, Loss=15.318681, Triplet loss=1.814214, Training Accuracy=0.406250, lr=0.001000
Iter=245760/epoch=0, Loss=19.298210, Triplet loss=1.814214, Training Accuracy=0.421875, lr=0.001000
Iter=250880/epoch=0, Loss=14.741699, Triplet loss=1.814214, Training Accuracy=0.406250, lr=0.001000
Iter=256000/epoch=0, Loss=15.059827, Triplet loss=1.814214, Training Accuracy=0.500000, lr=0.001000
Iter=261120/epoch=0, Loss=18.078640, Triplet loss=1.792116, Training Accuracy=0.390625, lr=0.001000
Iter=266240/epoch=0, Loss=13.733654, Triplet loss=1.792116, Training Accuracy=0.500000, lr=0.001000
Iter=271360/epoch=0, Loss=15.891706, Triplet loss=1.814214, Training Accuracy=0.484375, lr=0.001000
Iter=276480/epoch=0, Loss=16.991713, Triplet loss=1.792116, Training Accuracy=0.453125, lr=0.001000
Iter=281600/epoch=0, Loss=13.313429, Triplet loss=1.747922, Training Accuracy=0.484375, lr=0.001000


Iter=640000/epoch=1, Loss=13.469604, Triplet loss=1.814214, Training Accuracy=0.531250, lr=0.001000
Iter=645120/epoch=1, Loss=19.581873, Triplet loss=1.814214, Training Accuracy=0.343750, lr=0.001000
Iter=650240/epoch=1, Loss=8.820726, Triplet loss=1.814214, Training Accuracy=0.500000, lr=0.001000
Iter=655360/epoch=1, Loss=11.039279, Triplet loss=1.814214, Training Accuracy=0.531250, lr=0.001000
Iter=660480/epoch=1, Loss=12.464891, Triplet loss=1.814214, Training Accuracy=0.390625, lr=0.001000
Iter=665600/epoch=1, Loss=10.391453, Triplet loss=1.814214, Training Accuracy=0.515625, lr=0.001000
Iter=670720/epoch=1, Loss=10.813535, Triplet loss=1.814214, Training Accuracy=0.453125, lr=0.001000
Iter=675840/epoch=1, Loss=8.179691, Triplet loss=1.814214, Training Accuracy=0.500000, lr=0.001000
Iter=680960/epoch=1, Loss=10.205665, Triplet loss=1.814214, Training Accuracy=0.437500, lr=0.001000
Iter=686080/epoch=1, Loss=10.897605, Triplet loss=1.814214, Training Accuracy=0.500000, lr=0.001000
It

Iter=1039360/epoch=2, Loss=8.261430, Triplet loss=1.814214, Training Accuracy=0.593750, lr=0.001000
Iter=1044480/epoch=2, Loss=6.615098, Triplet loss=1.814214, Training Accuracy=0.531250, lr=0.001000
Iter=1049600/epoch=2, Loss=9.466108, Triplet loss=1.792116, Training Accuracy=0.500000, lr=0.001000
Iter=1054720/epoch=2, Loss=12.183436, Triplet loss=1.814214, Training Accuracy=0.484375, lr=0.001000
Iter=1059840/epoch=2, Loss=7.794647, Triplet loss=1.792116, Training Accuracy=0.546875, lr=0.001000
Iter=1064960/epoch=2, Loss=7.163620, Triplet loss=1.814214, Training Accuracy=0.453125, lr=0.001000
Iter=1070080/epoch=2, Loss=11.078117, Triplet loss=1.792116, Training Accuracy=0.468750, lr=0.001000
Iter=1075200/epoch=2, Loss=9.477226, Triplet loss=1.814214, Training Accuracy=0.484375, lr=0.001000
Iter=1080320/epoch=2, Loss=8.741299, Triplet loss=1.814214, Training Accuracy=0.390625, lr=0.001000
Iter=1085440/epoch=2, Loss=10.059561, Triplet loss=1.814214, Training Accuracy=0.484375, lr=0.0010

Iter=1443840/epoch=3, Loss=6.907715, Triplet loss=1.814214, Training Accuracy=0.640625, lr=0.001000
Iter=1448960/epoch=3, Loss=13.910169, Triplet loss=1.814214, Training Accuracy=0.453125, lr=0.001000
Iter=1454080/epoch=3, Loss=12.363459, Triplet loss=1.814214, Training Accuracy=0.515625, lr=0.001000
Iter=1459200/epoch=3, Loss=10.497819, Triplet loss=1.725809, Training Accuracy=0.625000, lr=0.001000
Iter=1464320/epoch=3, Loss=12.723995, Triplet loss=1.814214, Training Accuracy=0.562500, lr=0.001000
Iter=1469440/epoch=3, Loss=10.219685, Triplet loss=1.814214, Training Accuracy=0.562500, lr=0.001000
Iter=1474560/epoch=3, Loss=13.552500, Triplet loss=1.814214, Training Accuracy=0.484375, lr=0.001000
Iter=1479680/epoch=3, Loss=11.237007, Triplet loss=1.814214, Training Accuracy=0.578125, lr=0.001000
Iter=1484800/epoch=3, Loss=14.199793, Triplet loss=1.814214, Training Accuracy=0.562500, lr=0.001000
Iter=1489920/epoch=3, Loss=13.722702, Triplet loss=1.703728, Training Accuracy=0.593750, lr=

...

Iter=3860480/epoch=10, Loss=0.001383, Triplet loss=1.615340, Training Accuracy=1.000000, lr=0.001000
Iter=3865600/epoch=10, Loss=0.000042, Triplet loss=1.593243, Training Accuracy=1.000000, lr=0.001000
Iter=3870720/epoch=10, Loss=0.003839, Triplet loss=1.482757, Training Accuracy=1.000000, lr=0.001000
Iter=3875840/epoch=10, Loss=0.000361, Triplet loss=1.615340, Training Accuracy=1.000000, lr=0.001000
Iter=3880960/epoch=10, Loss=0.009490, Triplet loss=1.615340, Training Accuracy=1.000000, lr=0.001000
Iter=3886080/epoch=10, Loss=0.000036, Triplet loss=1.659534, Training Accuracy=1.000000, lr=0.001000
Iter=3891200/epoch=10, Loss=0.000873, Triplet loss=1.681631, Training Accuracy=1.000000, lr=0.001000
Iter=3891200/epoch=10, Validation Accuracy=0.932100, Precision=0.881916, Recall=0.997800
Iter=3896320/epoch=10, Loss=0.000280, Triplet loss=1.571146, Training Accuracy=1.000000, lr=0.001000
Iter=3901440/epoch=10, Loss=0.000776, Triplet loss=1.637437, Training Accuracy=1.000000, lr=0.001000
It

Iter=4259840/epoch=11, Loss=0.000571, Triplet loss=1.571146, Training Accuracy=1.000000, lr=0.001000
Iter=4264960/epoch=11, Loss=0.000022, Triplet loss=1.725825, Training Accuracy=1.000000, lr=0.001000
Iter=4270080/epoch=11, Loss=0.000040, Triplet loss=1.549049, Training Accuracy=1.000000, lr=0.001000
Iter=4275200/epoch=11, Loss=0.014442, Triplet loss=1.615340, Training Accuracy=0.984375, lr=0.001000
Iter=4280320/epoch=11, Loss=0.000436, Triplet loss=1.725825, Training Accuracy=1.000000, lr=0.001000
Iter=4285440/epoch=11, Loss=0.000099, Triplet loss=1.482757, Training Accuracy=1.000000, lr=0.001000
Iter=4290560/epoch=11, Loss=0.000131, Triplet loss=1.703728, Training Accuracy=1.000000, lr=0.001000
Iter=4295680/epoch=11, Loss=0.000445, Triplet loss=1.440680, Training Accuracy=1.000000, lr=0.001000
Iter=4300800/epoch=11, Loss=0.000185, Triplet loss=1.593243, Training Accuracy=1.000000, lr=0.001000
Iter=4300800/epoch=11, Validation Accuracy=0.835300, Precision=0.752523, Recall=0.999200
It

Iter=4659200/epoch=12, Loss=0.000103, Triplet loss=1.615340, Training Accuracy=1.000000, lr=0.001000
Iter=4664320/epoch=12, Loss=0.000036, Triplet loss=1.549049, Training Accuracy=1.000000, lr=0.001000
Iter=4669440/epoch=12, Loss=0.000105, Triplet loss=1.526951, Training Accuracy=1.000000, lr=0.001000
Iter=4674560/epoch=12, Loss=0.000188, Triplet loss=1.549049, Training Accuracy=1.000000, lr=0.001000
Iter=4679680/epoch=12, Loss=0.000010, Triplet loss=1.637437, Training Accuracy=1.000000, lr=0.001000
Iter=4684800/epoch=12, Loss=0.000052, Triplet loss=1.637437, Training Accuracy=1.000000, lr=0.001000
Iter=4689920/epoch=12, Loss=0.000095, Triplet loss=1.571146, Training Accuracy=1.000000, lr=0.001000
Iter=4695040/epoch=12, Loss=0.000017, Triplet loss=1.482757, Training Accuracy=1.000000, lr=0.001000
Iter=4700160/epoch=12, Loss=0.000309, Triplet loss=1.549049, Training Accuracy=1.000000, lr=0.001000
Iter=4705280/epoch=12, Loss=0.000122, Triplet loss=1.482757, Training Accuracy=1.000000, lr

Iter=5053440/epoch=13, Loss=0.002603, Triplet loss=1.350175, Training Accuracy=1.000000, lr=0.001000
Iter=5058560/epoch=13, Loss=0.000412, Triplet loss=1.460660, Training Accuracy=1.000000, lr=0.001000
Iter=5063680/epoch=13, Loss=0.000651, Triplet loss=1.414771, Training Accuracy=1.000000, lr=0.001000
Iter=5068800/epoch=13, Loss=0.000004, Triplet loss=1.460660, Training Accuracy=1.000000, lr=0.001000
Iter=5073920/epoch=13, Loss=0.000003, Triplet loss=1.571146, Training Accuracy=1.000000, lr=0.001000
Iter=5079040/epoch=13, Loss=0.000041, Triplet loss=1.416466, Training Accuracy=1.000000, lr=0.001000
Iter=5084160/epoch=13, Loss=0.000037, Triplet loss=1.482726, Training Accuracy=1.000000, lr=0.001000
Iter=5089280/epoch=13, Loss=0.000062, Triplet loss=1.438563, Training Accuracy=1.000000, lr=0.001000
Iter=5094400/epoch=13, Loss=0.000011, Triplet loss=1.460660, Training Accuracy=1.000000, lr=0.001000
Iter=5099520/epoch=13, Loss=0.023233, Triplet loss=1.593243, Training Accuracy=0.984375, lr

Iter=5452800/epoch=14, Loss=0.000001, Triplet loss=1.438563, Training Accuracy=1.000000, lr=0.001000
Iter=5457920/epoch=14, Loss=0.000000, Triplet loss=1.438563, Training Accuracy=1.000000, lr=0.001000
Iter=5463040/epoch=14, Loss=0.000015, Triplet loss=1.460660, Training Accuracy=1.000000, lr=0.001000
Iter=5468160/epoch=14, Loss=0.000000, Triplet loss=1.305981, Training Accuracy=1.000000, lr=0.001000
Iter=5473280/epoch=14, Loss=0.000385, Triplet loss=1.504854, Training Accuracy=1.000000, lr=0.001000
Iter=5478400/epoch=14, Loss=0.000573, Triplet loss=1.482757, Training Accuracy=1.000000, lr=0.001000
Iter=5483520/epoch=14, Loss=0.000078, Triplet loss=1.637437, Training Accuracy=1.000000, lr=0.001000
Iter=5488640/epoch=14, Loss=0.000155, Triplet loss=1.703728, Training Accuracy=1.000000, lr=0.001000
Iter=5493760/epoch=14, Loss=0.000019, Triplet loss=1.350175, Training Accuracy=1.000000, lr=0.001000
Iter=5498880/epoch=14, Loss=0.000004, Triplet loss=1.460660, Training Accuracy=1.000000, lr